In [17]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy

## Extract CSVs into DataFrames

In [18]:
census_df = pd.read_csv("census_data_county.csv", encoding="utf-8")
census_df.head()

,state,county,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Unemployment Count,Race: White,Race: African American,Race: Hispanic or Latino,"US Citizen, born in US","US Citizen, by naturalization"
0,Alabama,Autauga County,55200,37.8,58786.0,29372.0,8422.0,1065.0,42437,10565,1528,53385,579
1,Alabama,Baldwin County,208107,42.8,55962.0,31203.0,21653.0,4343.0,179526,19764,9353,198875,3319
2,Alabama,Barbour County,25782,39.9,34186.0,18461.0,6597.0,918.0,12216,12266,1106,25064,280
3,Alabama,Bibb County,22527,39.9,45340.0,20199.0,2863.0,658.0,17268,5018,547,22083,129
4,Alabama,Blount County,57645,40.8,48695.0,22656.0,8220.0,909.0,55054,862,5261,54877,876


In [19]:
votes_df = pd.read_csv("Resources/president_county_candidate.csv", encoding="utf-8")
votes_df.head()

,state,county,candidate,party,votes
0,Alabama,Autauga County,Donald Trump,REP,19764
1,Alabama,Autauga County,Joe Biden,DEM,7450
2,Alabama,Autauga County,Jo Jorgensen,LIB,346
3,Alabama,Autauga County,Write-ins,WRI,79
4,Alabama,Baldwin County,Donald Trump,REP,83055


In [20]:
biden_df = pd.read_csv("Resources/biden_tweets.csv", encoding="utf-8")
biden_df.head()

,tweet,city,country,state
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,NaN,USA,Florida
1,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,NaN,USA,Montana
2,#censorship #HunterBiden #Biden #BidenEmails #...,NaN,USA,California
3,"In 2020, #NYPost is being #censorship #CENSORE...",Chicago,USA,Illinois
4,"FBI Allegedly Obtained Hunter Biden Computer, ...",NaN,USA,Kentucky


In [21]:
trump_df = pd.read_csv("Resources/trump_tweets.csv", encoding="utf-8")
trump_df.head()

,tweet,city,country,state
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,NaN,USA,Florida
1,"#Trump: As a student I used to hear for years,...",Portland,USA,Oregon
2,You get a tie! And you get a tie! #Trump ‘s ra...,Washington,USA,District of Columbia
3,@CLady62 Her 15 minutes were over long time ag...,NaN,USA,California
4,@DeeviousDenise @realDonaldTrump @nypost There...,NaN,USA,Ohio


### Transform Census DataFrame

In [22]:
# Rename the column headers
census_transformed = census_df.rename(columns={"state": "State",
                                              "county": "County",
                                               "Population": "Population",
                                               "Median Age": "Median_Age",
                                               "Household Income": "Median_Household_Income",
                                               "Per Capita Income": "Per_Capita_Income",
                                               "Poverty Count": "Poverty_Count",
                                               "Unemployment Count": "Unemployment_Count",
                                               "Race: White": "Population_White",
                                               "Race: African American": "Population_AfricanAmerican",
                                               "Race: Hispanic or Latino": "Population_Hispanic",
                                               "US Citizen, born in US": "US_Citizen_Birth",
                                               "US Citizen, by naturalization": "US_Citizen_Naturalised"
                                              })

census_transformed

,State,County,Population,Median_Age,Median_Household_Income,Per_Capita_Income,Poverty_Count,Unemployment_Count,Population_White,Population_AfricanAmerican,Population_Hispanic,US_Citizen_Birth,US_Citizen_Naturalised
0,Alabama,Autauga County,55200,37.8,58786.0,29372.0,8422.0,1065.0,42437,10565,1528,53385,579
1,Alabama,Baldwin County,208107,42.8,55962.0,31203.0,21653.0,4343.0,179526,19764,9353,198875,3319
2,Alabama,Barbour County,25782,39.9,34186.0,18461.0,6597.0,918.0,12216,12266,1106,25064,280
3,Alabama,Bibb County,22527,39.9,45340.0,20199.0,2863.0,658.0,17268,5018,547,22083,129
4,Alabama,Blount County,57645,40.8,48695.0,22656.0,8220.0,909.0,55054,862,5261,54877,876
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,Wyoming,Sweetwater County,44117,34.6,73008.0,32624.0,5237.0,1213.0,41084,354,7042,41202,707
3042,Wyoming,Teton County,23059,39.3,83831.0,53703.0,1619.0,210.0,20832,274,3433,20351,578
3043,Wyoming,Uinta County,20609,35.5,58235.0,27009.0,2552.0,614.0,19252,20,1884,19748,110
3044,Wyoming,Washakie County,8129,43.5,53426.0,27556.0,984.0,253.0,7293,1,1153,7994,42


### Transform Votes Dataframe

In [23]:
# Rename columns
votes_transformed = votes_df.rename(columns={"state": "State",
                                        "county": "County",
                                         "candidate": "Candidate",
                                         "party": "Party",
                                         "votes": "Votes"
                                        })
votes_transformed

,State,County,Candidate,Party,Votes
0,Alabama,Autauga County,Donald Trump,REP,19764
1,Alabama,Autauga County,Joe Biden,DEM,7450
2,Alabama,Autauga County,Jo Jorgensen,LIB,346
3,Alabama,Autauga County,Write-ins,WRI,79
4,Alabama,Baldwin County,Donald Trump,REP,83055
...,...,...,...,...,...
19439,Wyoming,Weston County,Donald Trump,REP,3107
19440,Wyoming,Weston County,Joe Biden,DEM,360
19441,Wyoming,Weston County,Jo Jorgensen,LIB,46
19442,Wyoming,Weston County,Brock Pierce,IND,20


### Transform Biden Dataframe

In [24]:
# Create a filtered dataframe from specific columns
biden_cols = ["tweet", "country", "state"]
biden_transformed = biden_df[biden_cols].copy()

# Rename the column headers
biden_transformed = biden_transformed.rename(columns={"tweet": "Tweet",
                                                     "country": "Country",
                                                      "state": "State"
                                                     })

biden_transformed.set_index("State", inplace=True)
biden_transformed.head()

,Tweet,Country
State,,
Florida,#Elecciones2020 | En #Florida: #JoeBiden dice ...,USA
Montana,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,USA
California,#censorship #HunterBiden #Biden #BidenEmails #...,USA
Illinois,"In 2020, #NYPost is being #censorship #CENSORE...",USA
Kentucky,"FBI Allegedly Obtained Hunter Biden Computer, ...",USA


### Transform Trump Dataframe

In [25]:
# Create a filtered dataframe from specific columns
trump_cols = ["tweet", "country", "state"]
trump_transformed = trump_df[trump_cols].copy()

# Rename the column headers
trump_transformed = trump_transformed.rename(columns={"tweet": "Tweet",
                                                     "country": "Country",
                                                      "state": "State"
                                                     })

trump_transformed.set_index("State", inplace=True)
trump_transformed.head()

,Tweet,Country
State,,
Florida,#Elecciones2020 | En #Florida: #JoeBiden dice ...,USA
Oregon,"#Trump: As a student I used to hear for years,...",USA
District of Columbia,You get a tie! And you get a tie! #Trump ‘s ra...,USA
California,@CLady62 Her 15 minutes were over long time ag...,USA
Ohio,@DeeviousDenise @realDonaldTrump @nypost There...,USA


### Create database connection

In [26]:
username = 'postgres'
password = 'postgres'
host = '35.197.164.165'
port = '5432'
database = 'postgres'

db_url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    username, password, host, port, database)

engine = sqlalchemy.create_engine(db_url)

conn = engine.connect()

In [27]:
# Confirm tables
engine.table_names()

['vote_data', 'biden_tweets', 'trump_tweets', 'census_data']

### Load DataFrames into Database (Grace's part)